In [1]:
import pandas as pd

In [13]:
def make_data(data_file,
             annotated_file,
             add_files):
    df_data = pd.read_csv(f'{data_file}')
    df_annotated = pd.read_csv(f'{annotated_file}')
    df_summaries = df_data[~df_data['id'].isin(df_annotated['docid'])]

    added_dfs = [df_summaries]
    for add_file in add_files:
        df_add = pd.read_csv(f'{add_file}')
        added_dfs += [df_add]
    return pd.concat(added_dfs)

In [14]:
# df_genaudit = pd.read_csv('/Users/srampras/hallucinations_annotations/scripts/data/genaudit_annotations.csv')


data_file = '/Users/ramprasad.sa/hallucinations_annotations_trial/data_new/llm_aggrefact_sample.csv'
annotated_file = '/Users/ramprasad.sa/hallucinations_annotations_trial/annotated_data/first_batch_final4_annotator_dump_oct_28.csv'
# add_files = ['llama3_8b_summaries.csv']
add_files = []

df_summaries = make_data(data_file = data_file,
                        annotated_file = annotated_file,
                        add_files = add_files)

In [15]:
len(df_summaries)

500

In [16]:
import psycopg2

In [22]:
conn = psycopg2.connect(
   database="d65103c5a2kbgs", 
    user='u16ifm924gr2m8', 
    password='pe4029f0ba1fbaafd034f4dbf17f703bf9a5d6c8420f07719f5db4054c9d4da40', 
    host='c1gvrf2q90nbcq.cluster-czrs8kj4isg7.us-east-1.rds.amazonaws.com', 
    port = '5432'
    
)
cursor = conn.cursor()

In [29]:
import re 
cursor.execute("""SELECT * from model_summaries""")
# print('Size before adding' , len(cursor.fetchall()))

# df_genaudit = df_genaudit[df_genaudit['faithful'] == 1]
df_summaries = df_summaries[~df_summaries['doc'].isnull()]
df_summaries = df_summaries[~df_summaries['claim'].isnull()]
for idx, row in df_summaries.iterrows():
    # print(row)
    docid = row['id']
    source = row['doc']
    summary = row['claim']
    model = 'unknown'
    origin = row['dataset']
    benchmark_dataset_name = row['dataset']
    if 'claim' in df_summaries.keys(): 
        # and row['DocID'] in filter_ids:
        summary = row['claim']
        summary_uuid_generic = f'{docid}_{benchmark_dataset_name}_{origin}_{model}'
        summary = re.sub( "'", r"''", summary)
        source = re.sub( "'", r"''", source)
        value_str = f"""'{docid}', '{source}', '{summary}', '{model}', '{benchmark_dataset_name}', '{origin}'"""
        # print(re.findall(r"\'(\w+)\'", dialogue))
        cursor.execute(f"""INSERT INTO model_summaries(docid, source, summary, model, benchmark_dataset_name, origin) VALUES ({value_str})""",
                                                        )
        
   

In [30]:
cursor.execute("""SELECT * from model_summaries""")
print('Size before adding' , len(cursor.fetchall()))

Size before adding 539


In [31]:
conn.commit()